### Tidy Data

Good practice is to import the necessary libraries at the start of your notebook/script

In [11]:
import pandas as pd
import numpy as np

Getting your data into this format requires some upfront work, but that work pays off in the long term. Once you have tidy data and pandas data frames, you will spend much less time manipulating data from one representation to another, allowing you to spend more time on the analytic questions at hand.

Instead of using a csv file, we can also connet to datasets via a URL and then create a dataframe using `pandas`

In [12]:
base_url = "https://github.com/byuidatascience/data4python4ds/raw/master/data-raw/" # base URL for datasets
table1 = pd.read_csv("{}table1/table1.csv".format(base_url)) # read table1 dataset
table2 = pd.read_csv("{}table2/table2.csv".format(base_url)) # read table2 dataset
table3 = pd.read_csv("{}table3/table3.csv".format(base_url)) # read table3 dataset
table4a = pd.read_csv("{}table4a/table4a.csv".format(base_url)) # read table4a dataset
table4b = pd.read_csv("{}table4b/table4b.csv".format(base_url)) # read table4b dataset
table5 = pd.read_csv("{}table5/table5.csv".format(base_url), dtype = 'object') # read table5 dataset

In [13]:

display(table1)
display(table2)
display(table3)
display(table4a)
display(table4b)


,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Afghanistan,2000,2666,20595360
2,Brazil,1999,37737,172006362
3,Brazil,2000,80488,174504898
4,China,1999,212258,1272915272
5,China,2000,213766,1280428583


,country,year,type,count
0,Afghanistan,1999,cases,745
1,Afghanistan,1999,population,19987071
2,Afghanistan,2000,cases,2666
3,Afghanistan,2000,population,20595360
4,Brazil,1999,cases,37737
5,Brazil,1999,population,172006362
6,Brazil,2000,cases,80488
7,Brazil,2000,population,174504898
8,China,1999,cases,212258
9,China,1999,population,1272915272


,country,year,rate
0,Afghanistan,1999,745/19987071
1,Afghanistan,2000,2666/20595360
2,Brazil,1999,37737/172006362
3,Brazil,2000,80488/174504898
4,China,1999,212258/1272915272
5,China,2000,213766/1280428583


,country,1999,2000
0,Afghanistan,745,2666
1,Brazil,37737,80488
2,China,212258,213766


,country,1999,2000
0,Afghanistan,19987071,20595360
1,Brazil,172006362,174504898
2,China,1272915272,1280428583


These are all representations of the same underlying data, but they are not equally easy to use. One dataset, the tidy dataset, will be much easier to work with.

There are three interrelated rules which make a dataset tidy:

**1. Each variable must have its own column.**

**2. Each observation must have its own row.**

**3. Each value must have its own cell.**

![GitHub Codespaces](tidy_data.png)

In this example, only table1 is tidy. It’s the only representation where each column is a variable.

Why ensure that your data is tidy? There are two main advantages:

1. There’s a general advantage to picking one consistent way of storing data. If you have a consistent data structure, it’s easier to learn the tools that work with it because they have an underlying uniformity.

2. There’s a specific advantage to placing variables in columns because it allows `Pandas’` and `NumPy’s` vectorised nature to shine. As you learned in `assign` and `aggregate` functions, most built-in functions work with vectors of values. That makes transforming tidy data feel particularly natural.

`Pandas` work well with tidy data. Here are a couple of small examples showing how you might work with table1.



In [14]:
# Compute rate per 10,000


# using lambda


### Pivoting

Untidy data will suffer from one of two common problems:

- One variable might be spread across multiple columns.

- One observation might be scattered across multiple rows.

Typically a dataset will only suffer from one of these problems; it’ll only suffer from both if you’re really unlucky! To fix these problems, you’ll need two functions in pandas: `melt()`, `pivot()`, and `pivot_table()`. There are two additional functions called `stack()` and `unstack()` that use multi-index columns and rows.

**1. Pivoting longer `.melt()`** - Sometimes some of the column names are not names of variables, but values of a variable. To tidy a dataset like this, we need to stack the offending columns into a new pair of variables.

In [15]:
table4a

,country,1999,2000
0,Afghanistan,745,2666
1,Brazil,37737,80488
2,China,212258,213766


To tidy a dataset like this, we need to stack the offending columns into a new pair of variables. To describe that operation we need three parameters:

 - The set of columns whose names are identifier variables, not values. In this example, `country` is the identifier column and the value columns are `1999` and `2000`.

- The name of the variable to move the column names to. Here it is `year`.

- The name of the variable to move the column values to. Here it’s `cases`.

In [16]:
table4a.melt(["country"], var_name="year", value_name="cases")

,country,year,cases
0,Afghanistan,1999,745
1,Brazil,1999,37737
2,China,1999,212258
3,Afghanistan,2000,2666
4,Brazil,2000,80488
5,China,2000,213766


#### Exercise

Use `.melt()` to tidy table4b in a similar fashion. 

**2. Pivoting wider `.pivot()`** - is the opposite of `melt()`. You use it when an observation is scattered across multiple rows. For example, take table2

In [17]:
table2

,country,year,type,count
0,Afghanistan,1999,cases,745
1,Afghanistan,1999,population,19987071
2,Afghanistan,2000,cases,2666
3,Afghanistan,2000,population,20595360
4,Brazil,1999,cases,37737
5,Brazil,1999,population,172006362
6,Brazil,2000,cases,80488
7,Brazil,2000,population,174504898
8,China,1999,cases,212258
9,China,1999,population,1272915272


To tidy this up, we first analyse the representation in similar way to `melt()`. This time, however, we only need two parameters:

The column to take variable names from. Here, it’s `type`.

The column to take values from. Here it’s `count`.

Depending on the index, we will need to use either `pivot()` or `pivot_table()`.

In [18]:
table2.pivot(
    index=["country","year"],
    columns = "type",
    values="count"
).reset_index()

type,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Afghanistan,2000,2666,20595360
2,Brazil,1999,37737,172006362
3,Brazil,2000,80488,174504898
4,China,1999,212258,1272915272
5,China,2000,213766,1280428583


Strict reshaping → requires that the combination of `index` + `columns` is unique.

If duplicates exist (e.g., if table2 had two rows for Afghanistan–1999–cases), `.pivot()` will throw an error, as we saw.

Let's create a dummy table and see what this looks like:

In [19]:
data = {
    "country": ["Afghanistan","Afghanistan","Afghanistan","Afghanistan","Brazil","Brazil"],
    "year":    [1999,1999,1999,1999,1999,1999],
    "type":    ["cases","cases","population","population","cases","population"],
    "count":   [745,750,19987071,19987071,37737,172006362]
}

table2a_example = pd.DataFrame(data)
table2a_example

,country,year,type,count
0,Afghanistan,1999,cases,745
1,Afghanistan,1999,cases,750
2,Afghanistan,1999,population,19987071
3,Afghanistan,1999,population,19987071
4,Brazil,1999,cases,37737
5,Brazil,1999,population,172006362


In [21]:
table2a_example.pivot_table(
    index=["country","year"],
    columns="type",
    values="count",
    aggfunc="sum").reset_index()

type,country,year,cases,population
0,Afghanistan,1999,1495,39974142
1,Brazil,1999,37737,172006362


By default, the resultant values will be the average of the replicates, we can take care of this by explicitly stating what summary statistic we would like to see using `aggfunc=''`

In [ ]:
table2a_example.pivot_table()# column to take values from

#### Split

`str.split()` pulls apart one column into multiple columns, by splitting wherever a separator character appears. Take table3:

In [22]:
table3

,country,year,rate
0,Afghanistan,1999,745/19987071
1,Afghanistan,2000,2666/20595360
2,Brazil,1999,37737/172006362
3,Brazil,2000,80488/174504898
4,China,1999,212258/1272915272
5,China,2000,213766/1280428583


The rate column contains both cases and population variables, and we need to split it into two variables. `str.split()` takes the name of the column to split. The names of the columns to separate into can be names using `rename()`.

In [24]:
new_columns = (table3.rate.str.splt("/",expand=True),
               rename(column={0:"cases",1:"population"}))
new_columns
pd.concat([table3.drop(columns = 'rate'), new_columns.astype('float')], axis = 1) # axis =1 means concatenate columns

AttributeError: 'StringMethods' object has no attribute 'splt'

#### Unite

For two string series the inverse of `str.split()` can be done with `+`: it combines multiple columns into a single column. 

In [25]:
table5['new'] = table5["country"]+table5["year"]
table5

,country,century,year,rate,new
0,Afghanistan,19,99,745/19987071,Afghanistan99
1,Afghanistan,20,00,2666/20595360,Afghanistan00
2,Brazil,19,99,37737/172006362,Brazil99
3,Brazil,20,00,80488/174504898,Brazil00
4,China,19,99,212258/1272915272,China99
5,China,20,00,213766/1280428583,China00


In [ ]:
table5.assign()
table5

#### Exercise 1: From Long to Wide

Task: 

`ukbabynames` contains a listing of UK baby names occurring more than three times per year
between 1974 and 2020

The dataset contains columns like `year`, `sex`, `name`, and `n`.

Use `.pivot()` or `.pivot_table()` **(recall the unique index!)** to find the total number of babies born by sex in each year.

Rows should be years, columns should be sex (M / F), and values should be the sum of counts.

**Challenge - calculate the percent of babies who are female using your now tidy dataset**

In [ ]:
# Import the dataset as gapminder using pd.read_csv()
babynames = pd.read_csv('../Data/ukbabynames.csv')
babynames
# babynames.info()

In [ ]:
babies_by_sex = 

babies_by_sex

# This is now a tidy long dataframe, I could apply Pythons vectorised nature and easily calculate percent female babies


#### Exercise 2: From Wide to Long

WHO Tuberculosis Data

This dataset comes from the World Health Organization (WHO) and records the number of **tuberculosis (TB) cases** reported by different countries, broken down by sex and age group.

- **country, iso2, iso3**: identifiers for each country (name and ISO codes).
- **year**: the year of observation.
- **new_sp_m014, new_sp_m1524, …**: the number of new TB cases, where the code describes the **type of case**, the **sex**, and the **age group**.
  - Example: `new_sp_m014` = new cases, sputum positive (`sp`), male (`m`), ages 0–14 (`014`).


In [ ]:
# Import the dataset as gapminder using pd.read_csv()
who = pd.read_csv('../Data/who.csv')
who
who.info()

#### Exercise: Reshape WHO Tuberculosis Data

The WHO dataset contains TB cases reported by country and year. Each combination of **type**, **sex**, and **age group** is stored as a separate column (`new_sp_m014`, `new_sp_f1524`, etc.), which makes the dataset **wide**.

Your task:  
1. Use `pd.melt()` to convert columns 5 through 60 into **two new columns**:  
   - `tb_codes` → the original variable name (e.g., `new_sp_m014`).  
   - `n` → the number of TB cases.  

In [ ]:
# 2) Melt columns 5:60 into key-value pairs
who_long = who.melt()
who_long

2. Clean the new tidy dataframe so it does not have missing `missing values` in the `n` column.

In [ ]:
# Drop missing values


# display the df


### All Done!